#### catatan: 
beberapa data taksonomi memang kosong dari NCBInya, misal data interaksi virus pycv malah terhubung ke genus capsicum bukan spesies capsicum. dan itu memang dari data GloBInya langsung

In [2]:
# # reset import package
# def reloadPackageOwn():
#     from importlib import reload  
#     import os # we use os.path.join, os.path.basename
#     import sys # we use sys.path
#     import glob # we use glob.glob
#     import importlib # we use importlib.import_module

#     import_folder = os.getcwd()
#     sys.path.append(import_folder) # this tells python to look in `import_folder` for imports
#     for src_file in glob.glob(os.path.join(import_folder, '*.py')):
#         name = os.path.basename(src_file)[:-3]
#         importlib.import_module(name)
#         reload(sys.modules[name])
#         importlib.import_module(name)
        
# reloadPackageOwn()

In [3]:
from tqdm import tqdm
from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.graphs import KG, Vertex
from pyrdf2vec.embedders import FastText,Word2Vec
from pyrdf2vec.walkers import RandomWalker
from pyvis.network import Network
from sklearn.manifold import TSNE
from umap import UMAP
from SPARQLWrapper import SPARQLWrapper
from jcopml.plot import plot_missing_value

import plotly.express as px
import pandas as pd
import numpy as np
import requests
import os
import networkx as nx
import matplotlib.pyplot as plt

from modul.vectorReferenced import get_taxon_vector,cek_ncbi_id_by_wiki_id_via_string
from modul.filterNodeEdge import removeNodeAndEdgeByFilter,removeEdgesNotInNodes
from modul.helper_umum import contains_string_entire_column,contains_string_entire_column_boolean, minmax, std_scale, contains_set_of_strings_entire_column
#from process import cek_bfs, nx_to_pyviz
from modul.grafHelper import _set_networkx_graph, _plot_nx_by_matplotlib
from modul.visualisasiHelper import embeddingPlot,plotly_graph
from modul.embeddingHelper import df_serangga_to_rdf, rdf_KG_to_embeddings, df_to_dictionary_taxon
from modul.custom_degree_centrality import degree_centrality_custom

In [4]:
ncbi_ontology_url = 'http://localhost:3030/mydataset/query'
dict(get_taxon_vector("Pepper yellow leaf curl virus",ncbi_ontology_url,False))['family']

'NCBI:10811_Geminiviridae'

#### Parameter

In [5]:
data=[
    ('begomovirus_contoh_hasil','Pepper yellow leaf curl virus','Aleyrodidae','Bemisia Tabaci'),
    ('1cucu','Cucumber mosaic virus','Aphididae','Myzus persicae'),
    ('2cri','Tomato chlorosis virus','Aleyrodidae','Bemisia Tabaci'),
    ('3wai','Maize chlorotic dwarf virus','Cicadellidae','Graminella nigrifrons'),
    ('4beg','Tomato yellow leaf curl China virus','Aleyrodidae','Bemisia Tabaci'),
    ('5pol','Cereal yellow dwarf virus','Aphididae','Schizaphis graminum'),
    ('6pea','Pea enation mosaic virus 1','Aphididae','Acyrthosiphon pisum'),
    ('7cucur','Cucurbit yellow stunting disorder virus','Aleyrodidae','Bemisia Tabaci'),
    ('8ten','Rice stripe tenuivirus','Delphacidae','Laodelphax striatellus'),
    ('9fiji','Southern rice black-streaked dwarf virus','Delphacidae','Sogatella furcifera'),
    ('10capchlo','Capsicum chlorosis orthotospovirus','Thripidae','Thrips Palmi'),
    ('11barley','Barley yellow dwarf virus GAV','Aphididae','Sitobion avenae'),
    ('12tospot','Tomato spotted wilt orthotospovirus','Thripidae','Frankliniella occidentalis'),
    ('13svyv','squash vein yellowing virus','Aleyrodidae','Bemisia Tabaci'),
    ('14sbmv','soybean mosaic virus','Aphididae','Aphis glycines'),
    ('15blv','bean leafroll virus','Aphididae','Acyrthosiphon pisum'),
    ('16rgdv','rice gall dwarf virus','Cicadellidae','Recilia dorsalis'), #sedikit
    ('17srbsdv','southern rice black-streaked dwarf virus','Delphacidae','Sogatella furcifera'),
    ('18tsrv','tomato severe rugose virus','Aleyrodidae','Bemisia tabaci'),
    ('19gbnv','groundnut bud necrosis virus','Thripidae','Thrips palmi'),
    ('20wbnv','Watermelon bud necrosis virus','Thripidae','Thrips palmi'),
    # error dibawah ini
    # ('+13Poty','Potyvirus','Aphididae','Myzus'),
    # ('+11tung','Tungrovirus','Nilaparvata','Nilaparvata'),
]

data_,nama_virus,acuan_,ujian_=data[0] # vektor acuan  #data virus
# link enpoint sparql ncbi_ontology
ncbi_ontology_url = 'http://localhost:3030/mydataset/query'

#### input data

In [6]:
#1
#baca data
df_node=pd.read_csv('dari_praproses/'+data_+'_node.csv',index_col=0) 
df_edge=pd.read_csv('dari_praproses/'+data_+'_edge.csv',index_col=0)

In [7]:
# df_node[df_node['group']=='virus']
# df_node['group'].unique()

contains_string_entire_column(df_node,'67753_Crinivirus').taxon_id.to_list()

['NCBI:267970',
 'NCBI:52135',
 'NCBI:67754',
 'NCBI:31713',
 'NCBI:72750',
 'NCBI:81931',
 'NCBI:558690',
 'NCBI:51330',
 'NCBI:67753']

In [8]:
acuan_

'Aleyrodidae'

In [9]:
# pra-proses khusus proses
# hapus serangga yg cuma famili (mengikuti acuan). soalnya klo cuma tampil famili apa gunanya?
filter_genus_sampai_species_null=(
    (df_node.genus.isnull()) &
    (df_node.species.isnull()) &
    (df_node['class']=='NCBI:50557_Insecta')
)
df_node,df_edge = removeNodeAndEdgeByFilter(df_node[filter_genus_sampai_species_null], df_node,df_edge)

removeNodeAndEdgeByFilter
sebelum : 2667 5101
sesudah : 2641 4989


In [10]:
print(len(df_edge))
df_edge.drop_duplicates(inplace=True)
print(len(df_edge))

4989
4989


In [11]:
# pra-proses khusus proses
#isi data kosong. mengisi takson kosong, dengan takson sebelumnya, untuk tambalan
takson=[
    'superkingdom','kingdom','phylum','class','order','family','genus','species'
]

for x,i in enumerate(takson):
    if (i!='superkingdom'): #selain superkingdom update dengan data sebelumnya
        for idx, row in df_node[pd.isnull(df_node[i])].iterrows():
            df_node.loc[idx,[i]] = row[takson[x-1]]+'^'+i
    else: 
        for idx, row in df_node[pd.isnull(df_node[i])].iterrows():
            df_node.loc[idx,[i]] = row[takson[x+1]]+'^'+i

In [12]:
# pra-proses khusus proses
# eksperimen tambahan. bikin fakta tambahan, yaitu relasi virus utama dengan acuan 
# virus_utama=df_node[df_node.virus_utama==True].taxon_id.to_list()
# serangga_acuan=contains_string_entire_column(df_node,acuan_).taxon_id.to_list()
# print(len(df_edge))
# for i in virus_utama:
#     for j in serangga_acuan:
#         dict = {'source_taxon_id':i,'target_taxon_id':j,'interaction_type':'pathogenOf'}
#         df_edge = pd.concat([pd.DataFrame(dict,index=[0]), df_edge], ignore_index = True)
#         # df_edge.loc[len(df_edge.index),['source_taxon_id','target_taxon_id','interaction_type']] = [i,j,'pathogenOf']
# print(len(df_edge))

#satu virus saja #perlu ini soalnya kalo tida 2cri nol
# virus_utama=df_node[df_node.virus_utama==True].taxon_id.to_list()
# serangga_acuan=contains_string_entire_column(df_node,acuan_).taxon_id.to_list()
# print(len(df_edge))
# for j in serangga_acuan:
#     dict_serangga = {'source_taxon_id':virus_utama[0],'target_taxon_id':j,'interaction_type':'pathogenOf'}
#     df_edge = pd.concat([pd.DataFrame(dict_serangga,index=[0]), df_edge], ignore_index = True)
#     # df_edge.loc[len(df_edge.index),['source_taxon_id','target_taxon_id','interaction_type']] = [i,j,'pathogenOf']
# print(len(df_edge))


In [13]:
# Ini harusnya di praproses. tapi belum fix baiknya hapus atau tidak
# hapus yang bukan virus utama, terakhir akurasi 0.85, kalau berkurang hapus saja ini
# bukan_virus_utama=(df_node['group']=="virus") & (df_node.virus_utama!=True)
# df_node,df_edge = removeNodeAndEdgeByFilter(df_node[bukan_virus_utama], df_node,df_edge)

In [14]:
if(len(df_node[df_node['group']=="serangga"])<=2):
    print("cuma dua serangga")

# Konversi graf

In [15]:
#3
#konversi graph 
gnx = _set_networkx_graph(df_node, df_edge)

# # cuma tampilan, visualisasi graf
# _plot_nx_by_matplotlib(gnx)

# visualisasi Graf

In [16]:
plotly_graph(gnx)

# # # cuma tampilan, visualisasi pyviz
# from pyvis.network import Network
# nt = Network('500px', '900px',directed=True,notebook=True)
# # nt.show_buttons(filter_=['physics'])
# nt.toggle_physics(True)

# for i,data in df_node.iterrows():
#     nt.add_node(
#         data['taxon_id'],
#         label=data['taxon_name'],
#         superkingdom=data['superkingdom'],
#         kingdom=data['kingdom'],
#         filum=data['phylum'],
#         kelas=data['class'],
#         ordo=data['order'],
#         famili=data['family'],
#         genus=data['genus'],
#         spesies=data['species'],
#         # group=data['group'],
#         color=data['color'],
#         )
    
# for i,data in df_edge.iterrows():
#     nt.add_edge(
#         data['source_taxon_id'],
#         data['target_taxon_id'],
#         label=data['interaction_type'])

# # nt.show_buttons(filter_=['physics'])
# nt.show("tmp.fig02.html")

In [17]:
# visualisasi embedding serangga

# embedding
# input : df_node, URL
URL = "http://pyRDF2Vec"
df_serangga = df_node[df_node['group']=="serangga"]
CUSTOM_KG = df_serangga_to_rdf(df_serangga, URL)
# proses
list_serangga=df_node[df_node['group']=="serangga"].taxon_id.to_list()
list_of_entities = [ URL+"#"+taxon_id for taxon_id in list_serangga ]
transformer, embeddings, _ = rdf_KG_to_embeddings(CUSTOM_KG, list_of_entities)
# sama dua2nya # list_of_entities == transformer._entities 

# dictionary serangga
dictionary_serangga = df_to_dictionary_taxon(df_serangga)
# output
# embeddings, list_of_entities, dictionary_serangga

# plotting
embeddingPlot(embeddings, list_of_entities, dictionary_serangga, 650, 650)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


# Analisis Interaksi

In [18]:
# from modul.custom_degree_centrality import degree_centrality_custom
# import importlib, sys
# importlib.reload(sys.modules['modul.custom_degree_centrality'])

#4 
# Degree Centrality Custom
virus_utama_ids=list(df_node[df_node['virus_utama']==True].taxon_id)
serangga_ids=list(df_node[df_node['group']=="serangga"].taxon_id)
# G=gnx
# if len(G) <= 1:
#     results_dc = {n: 1 for n in G}
# s = 1.0 / (len(G) - 1.0)
# results_dc = {}
# for n, d in G.degree(serangga_ids):
#     results_dc[n] = d * s
# nx.degree_centrality(gnx)
results_dc = degree_centrality_custom(gnx,virus_utama_ids,serangga_ids)
allnodes = gnx.nodes

Papilio thoas NCBI:262664 | degree: 2 | langsung: 0 | melalui tanaman: 1
Epitrix subcrinita NCBI:1191541 | degree: 4 | langsung: 0 | melalui tanaman: 86
Neoleucinodes elegantalis NCBI:1511177 | degree: 3 | langsung: 0 | melalui tanaman: 85
Spodoptera littoralis NCBI:7109 | degree: 10 | langsung: 0 | melalui tanaman: 110
Autographa gamma NCBI:254363 | degree: 6 | langsung: 0 | melalui tanaman: 97
Spodoptera litura NCBI:69820 | degree: 10 | langsung: 0 | melalui tanaman: 127
Diabrotica speciosa NCBI:133126 | degree: 4 | langsung: 0 | melalui tanaman: 13
Helicoverpa armigera NCBI:29058 | degree: 12 | langsung: 0 | melalui tanaman: 111
Tetragonisca angustula NCBI:166442 | degree: 6 | langsung: 0 | melalui tanaman: 9
Frieseomelitta varia NCBI:561572 | degree: 5 | langsung: 0 | melalui tanaman: 13
Scaptotrigona hellwegeri NCBI:597186 | degree: 1 | langsung: 0 | melalui tanaman: 1
Thrips hawaiiensis NCBI:163894 | degree: 2 | langsung: 0 | melalui tanaman: 24
Bathrips melanicornis NCBI:1520562

In [19]:
urutan = sorted(results_dc.items(), key=lambda item: item[1], reverse=True)
urutan

[('NCBI:7038', 3.3416611643264773e-06),
 ('NCBI:355109', 0.0),
 ('NCBI:262664', 0.0),
 ('NCBI:355026', 0.0),
 ('NCBI:340608', 0.0),
 ('NCBI:1191541', 0.0),
 ('NCBI:7139', 0.0),
 ('NCBI:1511177', 0.0),
 ('NCBI:7109', 0.0),
 ('NCBI:254363', 0.0),
 ('NCBI:69820', 0.0),
 ('NCBI:133126', 0.0),
 ('NCBI:29058', 0.0),
 ('NCBI:540886', 0.0),
 ('NCBI:1839606', 0.0),
 ('NCBI:355029', 0.0),
 ('NCBI:166442', 0.0),
 ('NCBI:57566', 0.0),
 ('NCBI:76712', 0.0),
 ('NCBI:561572', 0.0),
 ('NCBI:597186', 0.0),
 ('NCBI:115274', 0.0),
 ('NCBI:1137656', 0.0),
 ('NCBI:163894', 0.0),
 ('NCBI:1520562', 0.0),
 ('NCBI:1350418', 0.0),
 ('NCBI:407288', 0.0),
 ('NCBI:122869', 0.0),
 ('NCBI:224263', 0.0),
 ('NCBI:153012', 0.0),
 ('NCBI:163899', 0.0),
 ('NCBI:163893', 0.0),
 ('NCBI:133901', 0.0),
 ('NCBI:202456', 0.0),
 ('NCBI:30195', 0.0),
 ('NCBI:127405', 0.0),
 ('NCBI:124289', 0.0),
 ('NCBI:7459', 0.0),
 ('NCBI:156320', 0.0),
 ('NCBI:115078', 0.0),
 ('NCBI:132113', 0.0),
 ('NCBI:1228037', 0.0),
 ('NCBI:1634180', 0.0

In [20]:
#visualisasi data
# mengecek BFS Degree tertinggi
for to_search  in contains_string_entire_column(df_node,'NCBI:7460').taxon_id.to_list():#[urutan[0][0]]:#["NCBI:7038"]:#"NCBI:33377","NCBI:7036",
    for edge in nx.bfs_edges(gnx.to_undirected(), source=to_search, depth_limit=1):
            s_id, o_id = edge

            s_label = gnx.nodes[s_id]['label'] +' '+s_id
            o_label = gnx.nodes[o_id]['label'] +' '+o_id
            o_grup = gnx.nodes[o_id]['group']

            # skip subjek to_search
            # if (o_grup == 'serangga'):
            print(s_label,'-->', o_label)
    print('===============')

Apis mellifera NCBI:7460 --> Phaseolus vulgaris NCBI:3885
Apis mellifera NCBI:7460 --> Solanum lycopersicum NCBI:4081
Apis mellifera NCBI:7460 --> Sonchus arvensis NCBI:50192
Apis mellifera NCBI:7460 --> Alternanthera sessilis ex NCBI:221762
Apis mellifera NCBI:7460 --> Manihot esculenta NCBI:3983
Apis mellifera NCBI:7460 --> Leucaena leucocephala NCBI:3866
Apis mellifera NCBI:7460 --> Daucus carota NCBI:4039
Apis mellifera NCBI:7460 --> Brassica rapa var. rapa NCBI:3711
Apis mellifera NCBI:7460 --> Brassica rapa subsp. chinensis NCBI:93385
Apis mellifera NCBI:7460 --> Ageratum conyzoides NCBI:68299
Apis mellifera NCBI:7460 --> Asystasia gangetica NCBI:141292
Apis mellifera NCBI:7460 --> Lonicera japonica NCBI:105884
Apis mellifera NCBI:7460 --> Malva parviflora NCBI:145753
Apis mellifera NCBI:7460 --> Sidastrum paniculatum NCBI:108354
Apis mellifera NCBI:7460 --> Cucurbita pepo NCBI:3663
Apis mellifera NCBI:7460 --> Cucurbita maxima NCBI:3661
Apis mellifera NCBI:7460 --> Ipomoea tilia

In [21]:
# tidak wajib, cuma visualisasi
# visualisasi interaksi suatu node
target_filter = 'NCBI:7038'
filtered_edge=contains_string_entire_column(df_edge,target_filter)
substrings={target_filter}.union(
    set(filtered_edge.source_taxon_id.to_list()),
    set(filtered_edge.target_taxon_id.to_list())
)
filtered_node=contains_set_of_strings_entire_column(df_node,substrings)
filtered_gnx=_set_networkx_graph(filtered_node, filtered_edge)
# Mendapatkan simpul yang memiliki self-loop
isolated_nodes = [node for node, degree in dict(filtered_gnx.degree()).items() if degree == 0]
# Menghapus simpul-simpul tersebut
filtered_gnx.remove_nodes_from(isolated_nodes)
plotly_graph(filtered_gnx, True)

In [26]:
# tidak wajib, cuma visualisasi
count_tanaman_nodes = sum(1 for _, data in filtered_gnx.nodes(data=True) if 'group' in data and data['group'] == 'tanaman')
print("Jumlah node dengan data group=tanaman:", count_tanaman_nodes)
count_tanaman_nodes = sum(1 for _, data in filtered_gnx.nodes(data=True) if 'group' in data and data['group'] == 'serangga')
print("Jumlah node dengan data group=serangga:", count_tanaman_nodes)
count_tanaman_nodes = sum(1 for _, data in filtered_gnx.nodes(data=True) if 'group' in data and data['group'] == 'virus')
print("Jumlah node dengan data group=virus:", count_tanaman_nodes)

Jumlah node dengan data group=tanaman: 78
Jumlah node dengan data group=serangga: 11
Jumlah node dengan data group=virus: 13


In [22]:
# tidak wajib, cuma visualisasi
import plotly.graph_objects as go

grup_edge=filtered_edge.groupby('interaction_type').size().to_dict()
grup_edge.keys()
# Data dari grup_edge
labels = list(grup_edge.keys())
values = list(grup_edge.values())

# Hitung total data
total_data = sum(values)

# Hitung persentase masing-masing kategori
percentages = [(value / total_data) * 100 for value in values]

# Membuat diagram batang dengan warna yang bervariasi dan nilai jumlah
colors = ['#ec583e', '#04cb94', '#636cfb', '#ab63fa', '#ffa15a']  # Ganti dengan warna yang Anda inginkan
fig = go.Figure(data=[go.Bar(x=labels, y=values, marker_color=colors)])


# Menambahkan nilai jumlah pada tiap batang
for i, (value, percentage) in enumerate(zip(values, percentages)):
    fig.add_annotation(
        x=labels[i],
        y=value + 1,
        text=f"{value} ({percentage:.2f}%)",
        showarrow=False,
        font=dict(size=12, color='black'),  # Warna teks putih
        textangle=0,
    )

# Menambahkan label pada sumbu x dan y
fig.update_xaxes(title_text='Interaction Type')
fig.update_yaxes(title_text='Count')

# Menambahkan judul diagram
fig.update_layout(title_text='Diagram Batang Interaksi dengan Warna yang Bervariasi dan Nilai Jumlah')

# Menampilkan diagram
fig.show()

# Analisis Taksonomi

In [253]:
#5
# Ambil data NCBI
# data acuan
data_acuan=get_taxon_vector(acuan_,ncbi_ontology_url,False)
print(data_acuan)
data_ujian=get_taxon_vector(ujian_,ncbi_ontology_url)
print(data_ujian)

[('family', 'NCBI:7036_Aleyrodidae'), ('order', 'NCBI:7524_Hemiptera'), ('class', 'NCBI:50557_Insecta'), ('phylum', 'NCBI:6656_Arthropoda'), ('kingdom', 'NCBI:33208_Metazoa'), ('superkingdom', 'NCBI:2759_Eukaryota')]
[('spesies', 'NCBI:7038_Bemisia tabaci'), ('genus', 'NCBI:7037_Bemisia'), ('famili', 'NCBI:7036_Aleyrodidae'), ('ordo', 'NCBI:7524_Hemiptera'), ('kelas', 'NCBI:50557_Insecta'), ('filum', 'NCBI:6656_Arthropoda'), ('kingdom', 'NCBI:33208_Metazoa'), ('superkingdom', 'NCBI:2759_Eukaryota')]


In [254]:
import importlib, sys
importlib.reload(sys.modules['modul.visualisasiHelper'])
from modul.visualisasiHelper import embeddingPlot,plotly_graph

#6 #konversi node networkx ke RDF
# input : df_node, URL, data_acuan
URL = "http://pyRDF2Vec"
df_serangga = df_node[df_node['group']=="serangga"]
CUSTOM_KG = df_serangga_to_rdf(df_serangga, URL, data_acuan)

#7 #embedding
list_serangga=df_node[df_node['group']=="serangga"].taxon_id.to_list()
# list entity yang akan diembedd. serangga acuan urutan terakhir
list_of_entities = [ URL+"#"+taxon_id for taxon_id in list_serangga ]
list_of_entities.append(f"{URL}#SERANGGA_ACUAN")
transformer, embeddings, _ = rdf_KG_to_embeddings(CUSTOM_KG, list_of_entities)
# list_of_entities == transformer._entities # True # artinya sama dua2nya

# dictionary serangga
dictionary_serangga = df_to_dictionary_taxon(df_serangga)
# output
# embeddings, list_of_entities, dictionary_serangga



# visualisasi
# input
# embeddings, list_of_entities, dictionary_serangga
embeddingPlot(embeddings, list_of_entities, dictionary_serangga, 650, 650)

In [255]:
#8
#euclidean distance

# buat dataframe
data_to_count=pd.DataFrame(embeddings, columns=list(range(0,100)))

# buat kolom label
ent=[data['label'] for index,data in gnx.nodes(data=True) if(data['group']=='serangga')] #jika serangga
ent.append("#SERANGGA_ACUAN")
data_to_count['label']=ent

#buat kolom entity
data_to_count['entity']=[i.replace("http://pyRDF2Vec#","") for i in transformer._entities]

# buat kolom hasil dc
for idx,row in data_to_count.iterrows(): #jika serangga acuan maka DC di isi nilai 1
    data_to_count.loc[idx,['dc_result']] = results_dc[row['entity']] if(row['entity']!="SERANGGA_ACUAN") else 1

#ambil koordinat acuan
acuan=next(data_to_count[data_to_count['label']=='#SERANGGA_ACUAN'].iterrows())[1]
acuan=np.array(tuple(acuan[i] for i in range(0,100)))
acuan

#hitung ED
for idx, row in data_to_count.iterrows():
    temp = np.array(tuple(row[i] for i in range(0,100)))
    data_to_count.loc[idx,['ed_result']] = np.linalg.norm(temp - acuan)

#drop data acuan
data_to_count.drop(data_to_count[data_to_count.label=="#SERANGGA_ACUAN"].index,inplace=True)

In [256]:
#drop kolom embedding
data_to_count.drop(columns=list(range(0,100)), inplace=True)

In [257]:
# simple scaling ed_result
# data_to_count["ed_result_scaled"] = data_to_count["ed_result"] / data_to_count["ed_result"].max()
data_to_count['ed_result_scaled'] = minmax(data_to_count['ed_result'])


# scaling dc 
# # dengan std
# data_to_count['dc_result_scaled'] = data_to_count['dc_result'] - data_to_count['dc_result'].mode()[0]
# # data_to_count['dc_result_scaled'] = data_to_count['dc_result']
# # lagi dengan minmax
# data_to_count["dc_result_scaled"] = data_to_count["dc_result_scaled"] / data_to_count["dc_result_scaled"].max()

# Perform scaling using standard deviation
# data_to_count['dc_result_scaled'] = std_scale(data_to_count['dc_result'])
# data_to_count['dc_result_scaled'] = minmax(data_to_count['dc_result_scaled'])
if data_to_count['dc_result'].max() == data_to_count['dc_result'].min():
    data_to_count['dc_result_scaled'] = data_to_count['dc_result']
else:
    data_to_count['dc_result_scaled'] = minmax(data_to_count['dc_result'])

In [258]:
data_to_count

,label,entity,dc_result,ed_result,ed_result_scaled,dc_result_scaled
0,Quadrus lugubris,NCBI:355109,0.0,3.030665,0.634427,0.0
1,Papilio thoas,NCBI:262664,0.0,3.307731,0.698421,0.0
2,Ouleus,NCBI:355026,0.0,3.096421,0.649614,0.0
3,Pinnaspis piperis,NCBI:340608,0.0,1.993255,0.394814,0.0
4,Epitrix subcrinita,NCBI:1191541,0.0,3.389127,0.717221,0.0
...,...,...,...,...,...,...
1078,Eretmocerus eremicus,NCBI:272444,0.0,3.665287,0.781006,0.0
1079,Encarsia nigricephala,NCBI:121805,0.0,3.581373,0.761625,0.0
1080,Encarsia quaintancei,NCBI:32399,0.0,3.611854,0.768665,0.0
1081,Encarsia inaron,NCBI:121801,0.0,3.651735,0.777876,0.0


In [259]:
#9
#hitung kombinasi
# for idx, row in data_to_count.iterrows():
#     _dc = row['dc_result_scaled']
#     _ed=( (row['ed_result_scaled']) if row['ed_result_scaled']!=0 else 1)
#     data_to_count.loc[idx,['result']] = _dc/_ed

data_to_count['result'] = (1+data_to_count['dc_result_scaled']) / (1+data_to_count['ed_result_scaled'])

# simple scaling result (final/kombinasi)
data_to_count['result'] = data_to_count['result'] / data_to_count['result'].max()

# Pengujian

In [260]:
# DC
data_to_count=data_to_count.sort_values('dc_result',ascending=False).reset_index(drop=True)
data_to_count[['label','entity','dc_result']]

,label,entity,dc_result
0,Bemisia tabaci,NCBI:7038,0.000003
1,Quadrus lugubris,NCBI:355109,0.000000
2,Melissodes dentiventris,NCBI:599278,0.000000
3,Spragueia leo,NCBI:56574,0.000000
4,Toxoneuron viator,NCBI:2927559,0.000000
...,...,...,...
1078,Plebeina hildebrandti,NCBI:398135,0.000000
1079,Eufriesea fallax,NCBI:28644,0.000000
1080,Thrinchostoma torridum,NCBI:479790,0.000000
1081,Braunsapis bouyssoui,NCBI:175315,0.000000


In [261]:

# Pengujian dc
for urutan in range(0,3):
    takson=[i[0] for i in data_ujian if i[0] not in ["superkingdom","kingdom","filum","kelas"]]
    id_hasil=data_to_count.iloc[urutan].entity
    cek_hasil= { k:v for k,v in reversed(allnodes[id_hasil].items()) if k in takson }
    cek_ujian= { k:v for k,v in data_ujian if k in takson }
    # print(acuan_,'->', data_)
    # print('ujian ',cek_ujian)
    print('hasil ',cek_hasil)
    cek=0
    for i in reversed(takson):
        cekk=cek_hasil[i]==cek_ujian[i]
        cek+=cekk
        # print(i, cekk)
    print(cek/len(takson))

hasil  {'spesies': 'NCBI:7038_Bemisia tabaci', 'genus': 'NCBI:7037_Bemisia', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
1.0
hasil  {'spesies': 'NCBI:355109_Quadrus lugubris', 'genus': 'NCBI:355105_Quadrus', 'famili': 'NCBI:40093_Hesperiidae', 'ordo': 'NCBI:7088_Lepidoptera'}
0.0
hasil  {'spesies': 'NCBI:599278_Melissodes dentiventris', 'genus': 'NCBI:81082_Melissodes', 'famili': 'NCBI:7458_Apidae', 'ordo': 'NCBI:7399_Hymenoptera'}
0.0


In [262]:
# ED
data_to_count=data_to_count.sort_values('ed_result',ascending=True).reset_index(drop=True)
data_to_count[['label','entity','ed_result']]

,label,entity,ed_result
0,Trialeurodes vaporariorum,NCBI:88556,0.283891
1,Bemisia afer,NCBI:166114,0.284512
2,Aleurodicus dispersus,NCBI:267823,0.303712
3,Bemisia tabaci,NCBI:7038,0.348893
4,Bemisia emiliae,NCBI:7037,0.447252
...,...,...,...
1078,Dolerus aprilis,NCBI:27532,4.108910
1079,Franklinothrips vespiformis,NCBI:297892,4.114079
1080,Ceratopogon,NCBI:209707,4.139411
1081,Abracris flavolineata,NCBI:58570,4.143668


In [263]:
# Pengujian ed
for urutan in range(0,3):
    takson=[i[0] for i in data_ujian if i[0] not in ["superkingdom","kingdom","filum","kelas"]]
    id_hasil=data_to_count.iloc[urutan].entity
    cek_hasil= { k:v for k,v in reversed(allnodes[id_hasil].items()) if k in takson }
    cek_ujian= { k:v for k,v in data_ujian if k in takson }
    # print(acuan_,'->', data_)
    # print('ujian ',cek_ujian)
    print('hasil ',cek_hasil)
    cek=0
    for i in reversed(takson):
        cekk=cek_hasil[i]==cek_ujian[i]
        cek+=cekk
        # print(i, cekk)
    print(cek/len(takson))

hasil  {'spesies': 'NCBI:88556_Trialeurodes vaporariorum', 'genus': 'NCBI:88555_Trialeurodes', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
0.5
hasil  {'spesies': 'NCBI:166114_Bemisia afer', 'genus': 'NCBI:7037_Bemisia', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
0.75
hasil  {'spesies': 'NCBI:267823_Aleurodicus dispersus', 'genus': 'NCBI:30098_Aleurodicus', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
0.5


In [264]:
# final score kombinasi
data_to_count=data_to_count.sort_values('result',ascending=False).reset_index(drop=True)
data_to_count#[['label','dc_result','ed_result','result']]

,label,entity,dc_result,ed_result,ed_result_scaled,dc_result_scaled,result
0,Bemisia tabaci,NCBI:7038,0.000003,0.348893,0.015014,1.0,1.000000
1,Trialeurodes vaporariorum,NCBI:88556,0.000000,0.283891,0.000000,0.0,0.507507
2,Bemisia afer,NCBI:166114,0.000000,0.284512,0.000143,0.0,0.507434
3,Aleurodicus dispersus,NCBI:267823,0.000000,0.303712,0.004578,0.0,0.505194
4,Bemisia emiliae,NCBI:7037,0.000000,0.447252,0.037732,0.0,0.489054
...,...,...,...,...,...,...,...
1078,Dolerus aprilis,NCBI:27532,0.000000,4.108910,0.883471,0.0,0.269453
1079,Franklinothrips vespiformis,NCBI:297892,0.000000,4.114079,0.884664,0.0,0.269282
1080,Ceratopogon,NCBI:209707,0.000000,4.139411,0.890515,0.0,0.268449
1081,Abracris flavolineata,NCBI:58570,0.000000,4.143668,0.891499,0.0,0.268309


In [265]:
# Pengujian kombinasi
for urutan in range(0,3):
    takson=[i[0] for i in data_ujian if i[0] not in ["superkingdom","kingdom","filum","kelas"]]
    id_hasil=data_to_count.iloc[urutan].entity
    cek_hasil= { k:v for k,v in reversed(allnodes[id_hasil].items()) if k in takson }
    cek_ujian= { k:v for k,v in data_ujian if k in takson }
    # print(acuan_,'->', data_)
    # print('ujian ',cek_ujian)
    print('hasil ',cek_hasil)
    cek=0
    for i in reversed(takson):
        cekk=cek_hasil[i]==cek_ujian[i]
        cek+=cekk
        # print(i, cekk)
    print(cek/len(takson))

hasil  {'spesies': 'NCBI:7038_Bemisia tabaci', 'genus': 'NCBI:7037_Bemisia', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
1.0
hasil  {'spesies': 'NCBI:88556_Trialeurodes vaporariorum', 'genus': 'NCBI:88555_Trialeurodes', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
0.5
hasil  {'spesies': 'NCBI:166114_Bemisia afer', 'genus': 'NCBI:7037_Bemisia', 'famili': 'NCBI:7036_Aleyrodidae', 'ordo': 'NCBI:7524_Hemiptera'}
0.75


In [266]:
# Pengujian kombinasi
# cek=1
# takson=[i[0] for i in data_ujian if i[0] not in ["superkingdom","kingdom","filum","kelas"]]
# urutan=0
# while cek/len(takson)>0:
#     id_hasil=data_to_count.iloc[urutan].entity
#     cek_hasil= { k:v for k,v in reversed(allnodes[id_hasil].items()) if k in takson }
#     cek_ujian= { k:v for k,v in data_ujian if k in takson }
#     # print(acuan_,'->', data_)
#     # print('ujian ',cek_ujian)
#     print('hasil ', urutan ,cek_hasil)
#     cek=0
#     for i in reversed(takson):
#         cekk=cek_hasil[i]==cek_ujian[i]
#         cek+=cekk
#         # print(i, cekk)
#     print(cek/len(takson))
#     urutan+=1

# Visualisasi Hasil Analisis

In [267]:
import plotly.graph_objects as go

to_itter=data_to_count.sort_values('dc_result',ascending=False).reset_index(drop=True)[:10]

label=[]
degree=[]
for index,data in to_itter.sort_values('dc_result',ascending=True).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.dc_result)

fig = go.Figure(data=go.Bar(
    x=degree,
    y=label,
    orientation='h'
))

fig.update_layout(
    title='Degree centrality',
    xaxis_title='Degree',
    yaxis_title='Insect'
)
fig.update_xaxes(tickformat=".2e", hoverformat=".2e")

fig.show()

In [268]:
to_itter=data_to_count.sort_values('ed_result',ascending=True).reset_index(drop=True)[:10]

label=[]
degree=[]
for index,data in to_itter.sort_values('ed_result',ascending=False).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.ed_result)

fig = go.Figure(data=go.Bar(
    x=degree,
    y=label,
    orientation='h'
))

fig.update_layout(
    title='Euclidean distance',
    xaxis_title='Distance',
    yaxis_title='Insect'
)

fig.show()

In [269]:
import plotly.graph_objects as go

to_itter=data_to_count.sort_values('result',ascending=False).reset_index(drop=True)[:10]

label=[]
degree=[]
for index,data in to_itter.sort_values('result',ascending=True).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.result)

fig = go.Figure(data=go.Bar(
    x=degree,
    y=label,
    orientation='h'
))

fig.update_layout(
    title='final score',
    xaxis_title='Score',
    yaxis_title='Insect'
)

fig.show()

In [270]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the color for all bars
bar_color = '#636EFA'

# First plot
to_itter = data_to_count.sort_values('dc_result', ascending=False).reset_index(drop=True)[:10]
label = []
degree = []
for index, data in to_itter.sort_values('dc_result', ascending=True).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.dc_result)

fig = make_subplots(rows=2, cols=2, subplot_titles=('(a) Degree centrality', '(b) Euclidean distance', '(c) Final score', ''), vertical_spacing=0.1, horizontal_spacing=0.15)
fig.add_trace(go.Bar(
    x=degree,
    y=label,
    orientation='h',
    marker={'color':bar_color}  # Set the color for the bars
), row=1, col=1)

# Second plot
to_itter = data_to_count.sort_values('ed_result', ascending=True).reset_index(drop=True)[:10]
label = []
degree = []
for index, data in to_itter.sort_values('ed_result', ascending=False).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.ed_result)

fig.add_trace(go.Bar(
    x=degree,
    y=label,
    orientation='h',
    marker={'color':bar_color}  # Set the color for the bars
), row=1, col=2)

# Third plot
to_itter = data_to_count.sort_values('result', ascending=False).reset_index(drop=True)[:10]
label = []
degree = []
for index, data in to_itter.sort_values('result', ascending=True).reset_index(drop=True).iterrows():
    label.append(data.label)
    degree.append(data.result)

fig.add_trace(go.Bar(
    x=degree,
    y=label,
    orientation='h',
    marker={'color':bar_color}  # Set the color for the bars
), row=2, col=1)

# Hide the empty subplot
fig.update_layout(showlegend=False)
fig.update_annotations(visible=False, selector="title.text")

# Update layout for subplot titles
fig.update_layout(
    # title_text="Combined Plots",
    title_font={'size':24},
    title_x=0.5,
    title_y=0.95,
    title_xanchor='center',
    title_yanchor='top'
)

# Rotate y-axis labels
fig.update_yaxes(tickangle=45)

fig.update_layout(height=800, width=1200)
fig.show()


# Dibawah ini tidak termasuk

In [271]:
# #visualisasi data
# # mengecek BFS Degree tertinggi
# for edge in nx.bfs_edges(gnx.to_undirected(), source="NCBI:7038", depth_limit=1):
#         s_id, o_id = edge

#         s_label = gnx.nodes[s_id]['label'] +' '+s_id
#         o_label = gnx.nodes[o_id]['label'] +' '+o_id
#         o_grup = gnx.nodes[o_id]['group']

#         # skip subjek to_search

#         print(s_label,'-->', o_label)
# print('===============')

In [272]:
df_node[df_node['taxon_name'].str.contains('Thrips')]

,taxon_id,taxon_name,taxon_path,taxon_path_ids,taxon_path_rank,taxon_rank,superkingdom,kingdom,phylum,class,order,family,genus,species,group,color,virus_utama
381,NCBI:163894,Thrips hawaiiensis,Animalia | Arthropoda | Insecta | Thysanoptera...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:30262 |...,kingdom | phylum | class | order | family | ge...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:30262_Thysanoptera,NCBI:45053_Thripidae,NCBI:45057_Thrips,NCBI:163894_Thrips hawaiiensis,serangga,#b22222,NaN
383,NCBI:1350418,Thrips parvispinus,Animalia | Arthropoda | Insecta | Thysanoptera...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:30262 |...,kingdom | phylum | class | order | family | ge...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:30262_Thysanoptera,NCBI:45053_Thripidae,NCBI:45057_Thrips,NCBI:1350418_Thrips parvispinus,serangga,#b22222,NaN
795,NCBI:161013,Thrips palmi,Animalia | Arthropoda | Insecta | Thysanoptera...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:30262 |...,kingdom | phylum | class | order | family | ge...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:30262_Thysanoptera,NCBI:45053_Thripidae,NCBI:45057_Thrips,NCBI:161013_Thrips palmi,serangga,#b22222,NaN
1046,NCBI:161014,Thrips tabaci,Animalia | Arthropoda | Insecta | Thysanoptera...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:30262 |...,kingdom | phylum | class | order | family | ge...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:30262_Thysanoptera,NCBI:45053_Thripidae,NCBI:45057_Thrips,NCBI:161014_Thrips tabaci,serangga,#b22222,NaN
1047,NCBI:407017,Thrips nigropilosus,Animalia | Arthropoda | Insecta | Thysanoptera...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:30262 |...,kingdom | phylum | class | order | family | ge...,NaN,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:30262_Thysanoptera,NCBI:45053_Thripidae,NCBI:45057_Thrips,NCBI:407017_Thrips nigropilosus,serangga,#b22222,NaN
1667,NCBI:45057,Thrips malloti,Animalia | Arthropoda | Insecta | Thysanoptera...,GBIF:1 | NCBI:6656 | NCBI:50557 | NCBI:30262 |...,kingdom | phylum | class | order | family | ge...,genus,NCBI:2759_Eukaryota,NCBI:33208_Metazoa,NCBI:6656_Arthropoda,NCBI:50557_Insecta,NCBI:30262_Thysanoptera,NCBI:45053_Thripidae,NCBI:45057_Thrips,GBIF:4799940_Thripsmalloti,serangga,#b22222,NaN


In [273]:
df_edge.interaction_type.unique()

array(['pathogenOf', 'hasHost', 'pollinates', 'visitFlowersOf', 'visit'],
      dtype=object)

In [274]:
ini=df_edge[df_edge['interaction_type'].isin(['visitFlowersOf'])]
ini

,source_taxon_id,target_taxon_id,interaction_type
511,NCBI:57566,NCBI:13196,visitFlowersOf
512,NCBI:76712,NCBI:13196,visitFlowersOf
514,NCBI:597186,NCBI:16739,visitFlowersOf
515,NCBI:115274,NCBI:130391,visitFlowersOf
626,NCBI:77572,NCBI:4071,visitFlowersOf
...,...,...,...
4885,NCBI:1053773,NCBI:2653735,visitFlowersOf
4886,NCBI:714786,NCBI:2653735,visitFlowersOf
4887,NCBI:596841,NCBI:2653735,visitFlowersOf
4888,NCBI:517685,NCBI:2653732,visitFlowersOf


In [275]:
df_node[df_node['taxon_id'].isin(
    ini.source_taxon_id.to_list()+
    ini.target_taxon_id.to_list()
)]

,taxon_id,taxon_name,taxon_path,taxon_path_ids,taxon_path_rank,taxon_rank,superkingdom,kingdom,phylum,class,order,family,genus,species,group,color,virus_utama
0,NCBI:3983,Manihot esculenta,Plantae | Tracheophyta | Magnoliopsida | Malpi...,GBIF:6 | NCBI:58023 | NCBI:3398 | NCBI:3646 | ...,kingdom | phylum | class | order | family | ge...,NaN,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:3646_Malpighiales,NCBI:3977_Euphorbiaceae,NCBI:3982_Manihot,NCBI:3983_Manihot esculenta,tanaman,#1f922b,NaN
2,NCBI:112408,Dimorphotheca sinuata,Plantae | Viridaeplantae | Streptophyta | Trac...,EOL_V2:281 | NCBI:33090 | EOL_V2:11823577 | NC...,kingdom | subkingdom | infrakingdom | phylum |...,NaN,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:4209_Asterales,NCBI:4210_Asteraceae,NCBI:13490_Dimorphotheca,NCBI:112408_Dimorphotheca sinuata,tanaman,#1f922b,NaN
3,NCBI:50207,Sonchus oleraceus,Plantae | Tracheophyta | Magnoliopsida | Aster...,EOL_V2:281 | NCBI:58023 | EOL:283 | NCBI:4209 ...,kingdom | phylum | class | order | family | ge...,NaN,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:4209_Asterales,NCBI:4210_Asteraceae,NCBI:50190_Sonchus,NCBI:50207_Sonchus oleraceus,tanaman,#1f922b,NaN
16,NCBI:37495,Euphorbia pulcherrima,root | cellular organisms | Eukaryota | Viridi...,NCBI:1 | NCBI:131567 | NCBI:2759 | NCBI:33090 ...,null | null | superkingdom | kingdom | phylum ...,NaN,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:3646_Malpighiales,NCBI:3977_Euphorbiaceae,NCBI:3990_Euphorbia,NCBI:37495_Euphorbia pulcherrima,tanaman,#1f922b,NaN
18,NCBI:189211,Crassocephalum crepidioides,root | cellular organisms | Eukaryota | Viridi...,NCBI:1 | NCBI:131567 | NCBI:2759 | NCBI:33090 ...,null | null | superkingdom | kingdom | phylum ...,NaN,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:4209_Asterales,NCBI:4210_Asteraceae,NCBI:189210_Crassocephalum,NCBI:189211_Crassocephalum crepidioides,tanaman,#1f922b,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628,NCBI:55597,Ageratum corymbosum,Plantae | Tracheophyta | Magnoliopsida | Aster...,GBIF:6 | NCBI:58023 | NCBI:3398 | NCBI:4209 | ...,kingdom | phylum | class | order | family | ge...,genus,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:4209_Asterales,NCBI:4210_Asteraceae,NCBI:55597_Ageratum,GBIF:8026613_Ageratumcorymbosum,tanaman,#1f922b,NaN
2629,NCBI:2653735,Ageratum maritimum,Plantae | Tracheophyta | Magnoliophyta | Magno...,INAT_TAXON:47126 | NCBI:58023 | INAT_TAXON:471...,kingdom | subkingdom | phylum | class | order ...,NaN,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:4209_Asterales,NCBI:4210_Asteraceae,NCBI:55597_Ageratum,NCBI:2653735_Ageratum maritimum,tanaman,#1f922b,NaN
2630,NCBI:2653732,Ageratum corymbosum,Plantae | Tracheophyta | Angiospermae | Magnol...,INAT_TAXON:47126 | NCBI:58023 | INAT_TAXON:471...,kingdom | phylum | subphylum | class | order |...,NaN,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:4209_Asterales,NCBI:4210_Asteraceae,NCBI:55597_Ageratum,NCBI:2653732_Ageratum corymbosum,tanaman,#1f922b,NaN
2634,NCBI:3629,Tarasa capitata,Cellular organisms | Eukaryota | Viridiplantae...,EOL:6061725 | NCBI:2759 | NCBI:33090 | EOL_V2:...,null | null | kingdom | phylum | null | null |...,family,NCBI:2759_Eukaryota,NCBI:33090_Viridiplantae,NCBI:35493_Streptophyta,NCBI:3398_Magnoliopsida,NCBI:41938_Malvales,NCBI:3629_Malvaceae,EOL:5417947_Tarasa,EOL:5412066_Tarasacapitata,tanaman,#1f922b,NaN


In [276]:
contains_string_entire_column(df_edge,'NCBI:1341303')

,source_taxon_id,target_taxon_id,interaction_type


In [277]:
contains_string_entire_column(df_node,'NCBI:12315')

,taxon_id,taxon_name,taxon_path,taxon_path_ids,taxon_path_rank,taxon_rank,superkingdom,kingdom,phylum,class,order,family,genus,species,group,color,virus_utama
875,NCBI:12315,Tomato aspermy virus,root | Viruses | Riboviria | Orthornavirae | K...,NCBI:1 | NCBI:10239 | NCBI:2559587 | NCBI:2732...,null | superkingdom | null | kingdom | phylum ...,NaN,NCBI:10239_Viruses,NCBI:2732396_Orthornavirae,NCBI:2732406_Kitrinoviricota,NCBI:2732461_Alsuviricetes,NCBI:2732544_Martellivirales,NCBI:39740_Bromoviridae,NCBI:12304_Cucumovirus,NCBI:12315_Tomato aspermy virus,virus,#671f92,NaN
